<a href="https://colab.research.google.com/github/dominikmeyer95/academic-output/blob/feature%2Fadvanced_derivatives/advanced_derivatives/retail_options_trading.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Advanced Derivatives

* Group: Dominik Meyer, Sean Goedgeluk, Krzysztof Rentflejsz
* Assignment: Exploit retail investors' impact on option pricing 

# Key Ressources & Notes

**Tebaldi's opener:**
* Retail investors do not use options for hedging but for leveraging upside
* Thus, retail investors put price pressure on the call side of option markets
* This phenomenon becomes more severe the easier the access to derivatives becomes, e.g. via neobroker like Robinhood


**Notes on online articles:**
* https://qz.com/2043668/the-real-reason-options-trading-pushed-robinhood-shares-higher
* https://www.cboe.com/insights/posts/how-meme-stocks-impact-options-trading/

* In meme stocks, delta hedging does not work anymore, because the typical negative correlation of the assets in the hedging portfolio (options and the underlying stock) has turned. When a stock price goes up, its implied volatility typically goes down, since if a stock is going up, in theory it’s because investors have more certainty about its future cashflows and profits. Meme stocks flip this around, so that the equity may shoot up in price, but it’s reasonable to expect that this price increase is highly unstable, thus the implied volatility stays elevated. Investment banks hence hedge a short call by a long call position as this hedges the volatility risk as well.

**Notes on papers:**
* https://www.nhh.no/contentassets/2e16f7ca8332414a893cc34759004d60/retail-option_sep2022.pdf

* "*The effect of individual investors on implied volatility depends on the nature of their option trading. In an environment with imperfect market maker hedging and inventory risks, widespread uninformed purchasing of options  either calls or puts) will create upward pressure on prices that manifests as higher implied volatility. On the other hand, if retail investors tend to sell options, this will create downward pressure on prices and reduce implied volatility. In aggregate, retail investors are more likely to purchase options than write them, and net option purchases are strongest for short-dated out-of-the-money options.*"

* "*Our work contributes to several strands of literature. One area of research emphasizes the role of demand pressures on option markets. For example, Bollen and Whaley (2004) argue that buying pressure helps explain differences in the shape of the moneyness curves between index and stock options. Garleanu, Pedersen, and Poteshman (2009) models demand-pressure effects and finds that proxies for option demand are related to the moneyness smirk. [...] Our evidence documents the important role that speculative retail investor demand pressure can have on the implied volatility surface, which suggests caution is warranted when interpreting the implied volatility surface as reflecting information about underlying firm fundamentals.*"

* "*Our evidence documents the important role that speculative retail investor
demand pressure can have on the implied volatility surface, which suggests caution is warranted when interpreting the implied volatility surface as reflecting information about underlying firm fundamentals.*"

* https://www.math.kth.se/matstat/seminarier/reports/M-exjobb14/140909.pdf

* "*The SVI implied volatility model is a parametric model for stochastic implied volatility. The SVI is interesting because of the possibility to state explicit conditions on its parameters so that the model does not generate prices where static arbitrage opportunities can occur. Calibration of the SVI model to real market data requires non-linear optimization algorithms and can be quite time consuming.*"


**Ideas:**
* Have a look at implied volatility and its variation
* Have a look at the stock price and its variation
* Have a look at the rolling correlation between both
* Have a look at the volatility surface as it might be less skewed based on that information
* (Have a look at the delta Skew, which indicates the demand for downside versus upside potential in options)

# Table of Contents

In [5]:
# import required packages
import numpy as np
import pandas as pd
from datetime import datetime
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
!pip install yfinance
import yfinance as yf


In [4]:
# import and format wrds data
# wrds_data = pd.read_csv('https://raw.githubusercontent.com/dominikmeyer95/academic-output/feature/advanced_derivatives/advanced_derivatives/wrds_data.csv')
wrds_data = pd.read_csv('https://raw.githubusercontent.com/dominikmeyer95/academic-output/feature/advanced_derivatives/advanced_derivatives/wrds_data.csv')
wrds_data['date'] = [datetime.strptime(str(date),'%Y%m%d') for date in wrds_data['date']]
wrds_data['impl_volatility'] = wrds_data['impl_volatility']*100

# filter wrds data
wrds_data_filtered = wrds_data[(wrds_data['date'] >= '2010-01-01') & (wrds_data['days'] == 30) & (wrds_data['cp_flag'] == 'C')]
wrds_data_filtered


secid       date  days  forward_price  strike_price    premium  \
52373   108105 2010-01-04    30    1131.288162   1131.288162  22.928560   
52395   108105 2010-01-05    30    1134.807356   1134.807356  22.271595   
52417   108105 2010-01-06    30    1135.420482   1135.420482  22.017977   
52439   108105 2010-01-07    30    1139.962923   1139.962923  21.918238   
52461   108105 2010-01-08    30    1143.247389   1143.247389  21.073979   
...        ...        ...   ...            ...           ...        ...   
227211  143439 2021-12-27    30    1094.443894   1094.443894  79.493497   
227233  143439 2021-12-28    30    1088.966259   1088.966259  78.114524   
227255  143439 2021-12-29    30    1086.675048   1086.675048  77.617082   
227277  143439 2021-12-30    30    1070.815708   1070.815708  74.968875   
227299  143439 2021-12-31    30    1057.241401   1057.241401  73.657344   

        impl_volatility     delta     gamma       theta        vega cp_flag  \
52373           17.7260  0.509269  0.006915 -128.844484  129.321840       C   
52395           17.1646  0.508947  0.007119 -124.791908  129.726750       C   
52417           16.9599  0.508828  0.007201 -123.210738  129.797893       C   
52439           16.8158  0.508746  0.007234 -122.560622  130.317981       C   
52461           16.1215  0.508349  0.007524 -117.403112  130.696490       C   
...                 ...       ...       ...         ...         ...     ...   
227211          63.6184  0.536318  0.001990 -484.872189  124.608157       C   
227233          62.8268  0.535867  0.002026 -476.502163  123.997825       C   
227255          62.5571  0.535713  0.002039 -473.434739  123.742370       C   
227277          61.3142  0.535006  0.002111 -457.374142  121.955955       C   
227299          61.0138  0.534835  0.002149 -449.351105  120.415450       C   

       ticker  index_flag  
52373     SPX           1  
52395     SPX           1  
52417     SPX           1  
52439     SPX           1  
52461     SPX           1  
...       ...         ...  
227211   TSLA           0  
227233   TSLA           0  
227255   TSLA           0  
227277   TSLA           0  
227299   TSLA           0  

[8935 rows x 14 columns]

In [3]:
# plot implied volatility
fig = px.line(wrds_data_filtered, x='date', y='impl_volatility', color='ticker')
fig.update_layout(template="plotly_dark", title="<b>Implied Volatility</b>", title_x=0.5, showlegend=True)
fig.update_xaxes(title_text='time', showgrid=True, gridcolor="grey", ticks="outside", tickwidth=1, tickcolor='grey', ticklen=5, linecolor="grey")
fig.update_yaxes(title_text='implied volatility in %', showgrid=True, gridcolor="grey", ticks="outside", tickwidth=1, tickcolor='grey', ticklen=5, linecolor="grey")
fig.show()

In [4]:
# missing rights to download stock market data from wrds
# temporarily try to download from yahoo finance instead
# does not work for S&P500 and yahoo finance data is of poor quality in general
yf_data = pd.DataFrame()
tsla = yf.download('TSLA', start='2010-01-01',  end='2021-12-31')
yf_data["tsla"] = tsla["Adj Close"]
spx = yf.download('SPX', start='2010-01-01',  end='2021-12-31')
yf_data["spx"] = spx["Adj Close"]
yf_data

[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed


tsla      spx
Date                           
2010-06-29    1.592667  0.03714
2010-06-30    1.588667  0.03611
2010-07-01    1.464000      NaN
2010-07-02    1.280000  0.29980
2010-07-06    1.074000      NaN
...                ...      ...
2021-12-23  355.666656      NaN
2021-12-27  364.646667      NaN
2021-12-28  362.823334      NaN
2021-12-29  362.063324      NaN
2021-12-30  356.779999      NaN

[2898 rows x 2 columns]

In [5]:
# compute 30-days rolling correlation of stock return and implied volatility

In [8]:
# create implied volatility surface subplots (temp dummy)

# read data from a csv
z_data = pd.read_csv('https://raw.githubusercontent.com/plotly/datasets/master/api_docs/mt_bruno_elevation.csv')

# initialize figure with 2 subplots
fig = make_subplots(rows=1, cols=2, specs= [[{'is_3d': True}, {'is_3d': True}]])
fig.add_trace(go.Surface(z=z_data.values, showscale=False), row=1, col=1)
fig.add_trace(go.Surface(z=z_data.values, showscale=False), row=1, col=2)
fig.update_layout(template="plotly_dark", title_text='Implied Volatility Surface')
fig.show()